<a href="https://colab.research.google.com/github/saptarshidatta96/MTech_Sem3/blob/main/RTAS2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall opencv-python

Found existing installation: opencv-python 4.1.2.30
Uninstalling opencv-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/cv2/*
    /usr/local/lib/python3.7/dist-packages/opencv_python-4.1.2.30.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled opencv-python-4.1.2.30


In [2]:
!pip uninstall opencv-contrib-python

Found existing installation: opencv-contrib-python 4.1.2.30
Uninstalling opencv-contrib-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/opencv_contrib_python-4.1.2.30.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled opencv-contrib-python-4.1.2.30


In [3]:
!pip install opencv-contrib-python

     |████████████████████████████████| 66.6 MB 110 kB/s 


In [4]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from pylab import *
from numpy import *
from scipy import linalg

In [5]:
k = [[1698.873755, 0.000000, 971.7497705],
     [0.000000, 1698.8796645, 647.7488275],
     [0.000000, 0.000000, 1.000000]]

k = np.asarray(k)

In [6]:
k

array([[1.69887376e+03, 0.00000000e+00, 9.71749770e+02],
       [0.00000000e+00, 1.69887966e+03, 6.47748827e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [7]:
k_inv = np.linalg.inv(k)

In [8]:
k_inv

array([[ 5.88625257e-04,  0.00000000e+00, -5.71996458e-01],
       [ 0.00000000e+00,  5.88623209e-04, -3.81279994e-01],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [9]:
img1 = cv2.imread('/content/im1.jpg', 0)
img2 = cv2.imread('/content/im2.jpg', 0)

In [10]:
def find_correspondences(img1, img2):
  sift = cv2.xfeatures2d.SIFT_create()
  kp1, des1 = sift.detectAndCompute(img1,None)
  kp2, des2 = sift.detectAndCompute(img2,None)
  flann = cv2.FlannBasedMatcher(dict(algorithm = 0, trees = 5),dict(checks=50))
  matches = flann.knnMatch(des1,des2,k=2)
  pts1 = []
  pts2 = []

  # ratio test as per Lowe's paper
  j = 0
  for i,(m,n) in enumerate(matches):
    if m.distance < 0.8*n.distance:
        pts2.append((kp2[m.trainIdx].pt, 1))
        pts1.append(kp1[m.queryIdx].pt)
        j = j+1
        if j == 100:
          break
  pts1 = [list(tup)+[1] for tup in pts1]
  pts2 = [list(tup)+[1] for tup in pts2]
  pts1, pts2 = np.asarray(pts1), np.asarray(pts2)
  pts1 = np.float64(pts1)
  pts2 = np.float64(pts1)

  return pts1, pts2

In [11]:
pts1, pts2 = find_correspondences(img1, img2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [12]:
def convert_to_camera_coordinate_system(pts1, pts2, k):
  k_inv = np.linalg.inv(k)
  pts1 = [np.dot(k_inv, item) for item in pts1]
  pts2 = [np.dot(k_inv, item) for item in pts2]
  return pts1, pts2

In [19]:
def compute_essential(pts1, pts2):
  
  x1, x2 = convert_to_camera_coordinate_system(pts1, pts2, k)
  x1 = np.asarray(x1)
  x2 = np.asarray(x2)
  n = x1.shape[1]
  if x2.shape[1] != n:
    raise ValueError("Number of points don't match.")
 
  # build matrix for equations
  A = []
  for i in range(8):
      z = np.kron(pts2[i], pts1[i])
      A.append(z)
  A = np.asarray(A)
  #print(A.shape)

  U,S,V = linalg.svd(A)
  E = V[-1].reshape(3,3)
  # constrain F
  # make rank 2 by zeroing out last singular value
  Ua,Sa,Va = linalg.svd(E)
  
  sigma = (Sa[0]+Sa[1])/2
  Sa[0] = sigma
  Sa[1] = sigma
  Sa[2] = 0
  
  Ea = np.dot(Ua,np.dot(diag(Sa),Va))
  

  return sigma , Ea


In [20]:
sigma, E = compute_essential(pts1, pts2)

In [21]:
E

array([[-3.43233757e-22,  1.80399143e-08, -3.09152220e-06],
       [-1.80399142e-08, -1.65426508e-18,  7.07106781e-01],
       [ 3.09152221e-06, -7.07106781e-01, -7.18875573e-20]])

In [22]:
np.linalg.det(E)

5.40243782500198e-30

In [23]:
U,S,V = linalg.svd(E)

In [24]:
S

array([7.07106781e-01, 7.07106781e-01, 1.46432517e-21])

In [30]:
R_z_pos90 = [[0, -1, 0],
             [1, 0, 0],
             [0, 0, 1]]
R_z_pos90 = np.asarray(R_z_pos90)
R_z_neg90 = [[0, 1, 0],
             [-1, 0, 0],
             [0, 0, -1]]
R_z_neg90 = np.asarray(R_z_neg90)             
R1 = np.dot(U ,np.dot(np.transpose(R_z_pos90), V))
R2 = np.dot(U ,np.dot(np.transpose(R_z_neg90), V))
t_x_1 = np.dot(U ,np.dot(R_z_pos90 ,np.dot(diag([sigma, sigma ,0]), np.transpose(U))))
t_x_2 = np.dot(U ,np.dot(R_z_neg90 ,np.dot(diag([sigma, sigma, 0]), np.transpose(U))))

In [31]:
R1

array([[-1.00000000e+00, -8.74414526e-06, -5.10245824e-08],
       [-8.74414526e-06,  1.00000000e+00, -2.23079943e-13],
       [-5.10245826e-08, -2.23085367e-13,  1.00000000e+00]])

In [32]:
R2

array([[ 1.00000000e+00,  8.74414526e-06,  5.10245824e-08],
       [ 8.74414526e-06, -1.00000000e+00,  2.23079943e-13],
       [ 5.10245826e-08,  2.23085367e-13, -1.00000000e+00]])

In [33]:
t_x_1

array([[ 0.00000000e+00,  1.80399140e-08, -3.09152220e-06],
       [-1.80399140e-08, -3.47851431e-19,  7.07106781e-01],
       [ 3.09152220e-06, -7.07106781e-01,  7.95429881e-19]])

In [34]:
t_x_2

array([[ 0.00000000e+00, -1.80399140e-08,  3.09152220e-06],
       [ 1.80399140e-08,  3.47851431e-19, -7.07106781e-01],
       [-3.09152220e-06,  7.07106781e-01, -7.95429881e-19]])